### Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.odr import ODR, Model, Data, RealData
from scipy.optimize import curve_fit
import seaborn as sns
import uncertainties as uc
from uncertainties.umath import sqrt

### Formatting

In [3]:
plt.style.use('Solarize_Light2')   #Options
                                    #Solarize_Light2
                                    #_classic_test_patch
                                    #_mpl-gallery
                                    #_mpl-gallery-nogrid
                                    #bmh
                                    #classic
                                    #dark_background
                                    #fast
                                    #fivethirtyeight
                                    #ggplot
                                    #grayscale
                                    #seaborn-v0_8
                                    #seaborn-v0_8-bright
                                    #seaborn-v0_8-colorblind
                                    #seaborn-v0_8-dark
                                    #seaborn-v0_8-dark-palette
                                    #seaborn-v0_8-darkgrid
                                    #seaborn-v0_8-deep
                                    #seaborn-v0_8-muted
                                    #seaborn-v0_8-notebook
                                    #seaborn-v0_8-paper
                                    #seaborn-v0_8-pastel
                                    #seaborn-v0_8-poster
                                    #seaborn-v0_8-talk
                                    #seaborn-v0_8-ticks
                                    #seaborn-v0_8-white
                                    #seaborn-v0_8-whitegrid
                                    #tableau-colorblind10

plt.rcParams.update({

    'figure.dpi': 150,                 # Resolution of figures
    'font.size': 10,                   # Default font size
    'font.family': 'sans-serif',       # Font family
    'font.sans-serif': ['DejaVu Sans'],
    
    'axes.titlesize': 10,              # Title font size
    'axes.labelsize': 8,              # Axis label font size
    'axes.linewidth': 0.8,             # Width of the axis lines
    'axes.grid': True,                 # Show grid
    # 'grid.color': '#dddddd',
    'grid.linestyle': '--',            # Grid line style (dashed)
    'grid.linewidth': 0.5,             # Grid line width
    
    'xtick.labelsize': 8,             # X-axis tick label size
    'ytick.labelsize': 8,             # Y-axis tick label size
    'xtick.direction': 'in',           # X-axis tick direction
    'ytick.direction': 'in',           # Y-axis tick direction

    'lines.linewidth': 0.8,            # Line width for plots
     'lines.markersize': 0.3,             # Marker size for scatter plots
     'lines.marker': 'o',               # Default marker shape

    'legend.frameon': True,            # Frame around legend
    'legend.framealpha': 0.8,          # Transparency of legend frame
    'legend.fontsize': 9,             # Font size in legend
    'legend.loc': 'best',              # Default legend location
})

### Basic Functions

In [4]:

# Errors functions
def rSquared(f_model,x,y,Params=None):
        if Params is None:
            Params = None
            y_pred = f_model(x)
            ss_res = np.sum((y - y_pred) ** 2)
            ss_tot = np.sum((y - np.mean(y)) ** 2)
            R2 = 1 - (ss_res / ss_tot)
        else:
            y_pred = f_model(x, *Params)
            ss_res = np.sum((y - y_pred) ** 2)
            ss_tot = np.sum((y - np.mean(y)) ** 2)
            R2 = 1 - (ss_res / ss_tot)
        return R2
def R2(y,y_pred):
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    R2 = 1 - (ss_res / ss_tot)
    return R2

def Residuals(y_measured,y_modelled):
    return y_measured - y_modelled

def MeanSquareError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def RootMeanSquareError(y_measured,y_modelled):
    return np.sqrt(MeanSquareError(y_measured,y_modelled))

def MeanAbsError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum(np.abs(y_measured-y_modelled))))

def ConfidenceInterval(y_measured,y_modelled,confInt=95):
        if(confInt == 95):
            Z = 1.96
        elif(confInt == 90):
            Z = 1.645
        elif(confInt == 99):
            Z = 2.576
        return Z*np.sqrt((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def STDOfMean(stdArray):
    return (1/len(stdArray)*np.sqrt(np.sum(stdArray**2)))

# Non Linear Curve Fiting Function
def NonLinAnalysis(f_model, x, y, ParamsArr, weights):
    if ParamsArr is None:
        Params = None
        std = None 
        R2 = rSquared(f_model,x,y,Params)
    else:
        Params, cov = curve_fit(f_model, x, y, p0=ParamsArr, sigma=weights)
        std = np.sqrt(np.diag(cov))
        R2 = rSquared(f_model,x,y,Params)
    return Params, std, R2


# Other Functions
def rpm_to_rad(rpm):
    return rpm*2*np.pi/60

def UnpackUncArray(Array):
     value = [Array[i].nominal_value for i in range(len(Array))]
     std = [Array[i].std_dev for i in range(len(Array))]
     return value,std 
def SortByfirst(arr1, arr2):
    """
    Sorts both arrays by the values in arr1.
    if arr1 = x and arr2 = y, 
        this function sorts by x
    """
    c=np.array([arr1, arr2])
    sorted_c=c[:, np.argsort(c[0])]
    return sorted_c[0], sorted_c[1]

### Inputting and Formatting Data

In [5]:
#Inputting Data
Task1 = pd.read_csv('data/Task1.csv')
Task2 = pd.read_csv('data/Task2.csv')
Task3 = pd.read_csv('data/Task3.csv')
Task4 = pd.read_csv('data/Task4.csv')

m_t1 = uc.ufloat(47e-3,0.01e-3) #kg mass of weight used
m_disk = uc.ufloat(1.500,0.0001) #1.500 #kg mass of the disk
h_t1 = uc.ufloat(1,0.0005) #m height above floor
R_disk = uc.ufloat(125e-3,0.1e-3) #125e-3 #m radius of disk
Rb_t1 = 32.5e-3 #m radius of bobbin
Rbear = uc.ufloat(16e-3,1e-3) #m radius of bearing
g = 9.81 #m/s^2
Rw_t3 = 0.170 #m radius of torque applied

def ArrayWithStaticUnc(Array,unc):
    a = []
    b = []
    for i in range(len(Array)):
            a.append(uc.ufloat(Array[i],unc))
            b.append(unc)
    return np.array(a), np.array(b)
# Data Formatting and Inserting uncertainties
Task1['w'] = rpm_to_rad(Task1['RPM'])
Task1['w_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task1['RPM'],0.1)[0])))
Task1['w_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task1['RPM'],0.1)[1])))

Task1['Time_mit_unc'], Task1['Time_unc'] = ArrayWithStaticUnc(Task1["Time"],0.2)

Task2['Trial1_w'] = rpm_to_rad(Task2['Try I  RPM'])
Task2['Trial1_w_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task2['Try I  RPM'],0.1)[0])))
Task2['Trail1_w_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task2['Try I  RPM'],0.1)[1])))
Task2['Trial2_w'] = rpm_to_rad(Task2['Try II RPM'])
Task2['Trial2_w_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task2['Try II RPM'],0.1)[0])))
Task2['Trail2_w_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task2['Try II RPM'],0.1)[1])))
Task2['Trial3_w'] = rpm_to_rad(Task2['Try III RPM'])
Task2['Trial3_w_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task2['Try III RPM'],0.1)[0])))
Task2['Trail3_w_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task2['Try III RPM'],0.1)[1])))
Task2['Time_mit_unc'], Task2['Time_unc'] = ArrayWithStaticUnc(Task2["Time(s)"],0.2)



Task3['m1w0'] = rpm_to_rad(Task3['ω0 RPM'])
Task3['m1w0_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task3['ω0 RPM'],0.1)[0])))
Task3['m1w0_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task3['ω0 RPM'],0.1)[1])))

Task3['m1w1'] = rpm_to_rad(Task3['ω1 RPM'])
Task3['m1w1_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task3['ω1 RPM'],0.1)[0])))
Task3['m1w1_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task3['ω1 RPM'],0.1)[1])))
Task3['Time1_mit_unc'], Task3['Time1_unc'] = ArrayWithStaticUnc(Task3["time s"],0.2)

Task3['m2w0'] = rpm_to_rad(Task3['ω0 RPM.1'])
Task3['m2w0_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task3['ω0 RPM.1'],0.1)[0])))
Task3['m2w0_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task3['ω0 RPM.1'],0.1)[1])))

Task3['m2w1'] = rpm_to_rad(Task3['ω1 RPM.1'])
Task3['m2w1_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task3['ω1 RPM.1'],0.1)[0])))
Task3['m2w1_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task3['ω1 RPM.1'],0.1)[1])))
Task3['Time2_mit_unc'], Task3['Time2_unc'] = ArrayWithStaticUnc(Task3["time s.1"],0.2)


Task4['w0'] = rpm_to_rad(Task4['ω0 RPM'])
Task4['w0_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task4['ω0 RPM'],0.1)[0])))
Task4['w1'] = rpm_to_rad(Task4['ω1 RPM'])
Task4['w1_mit_unc'] = (rpm_to_rad((ArrayWithStaticUnc(Task4['ω1 RPM'],0.1)[0])))
Task4["α_mit_unc"] = ArrayWithStaticUnc(Task4['α(degrees)'],0.5)[0]
Task4['Time_mit_unc'], Task4['Time_unc'] = ArrayWithStaticUnc(Task4["t s"],0.2)


### Task 1

1. Determine the moment of inertia of the gyro disk from a measurement of the angular acceleration
for a known torque as well as from a measurement of the rotation speed.

The gyro axis is positioned horizontally and is fixed to the tripod stand by the additional stand rod.
The string is wound on the bobbin and a weight is fixed at the string end. Measure the time the
weight needs to reach the floor from a certain rest position; immediately afterwards measure the
rotation speed of the gyro disk. Repeat the experiment, average over ten values and calculate the
standard deviation. During the preparation of the experiment think of a way to determine the
moment of inertia from the data. Derive the corresponding equations. Compare the measured value
of the moment of inertia with the one calculated from mass and diameter. This is the component I3
of the moment of inertia tensor.

#### From a measurement of the angular acceleration for a known torque

$$I_3\alpha = M = rF$$
$$ m_za = m_zg - F$$
Finding $\alpha$
$$a = r\alpha \implies \alpha = \frac{a}{r}$$
$$ h = v_ot + \frac{1}{2}at^2 , v_0 = 0 \implies a=\frac{2h}{t^2}$$
$$ \implies \alpha = \frac{2h}{rt^2} $$ 

Finding $I_3$
$$I_3 = \frac{rF}{\alpha}$$
$$ F = m_z(g-a) = m_z(g-\frac{2h}{t^2})$$
$$\implies I_3 = m_zr^2(\frac{gt^2}{2h} - 1) $$ 

In [ ]:
# Booklet Equation
def I(time,w,m,g,r):
      return((m*g*r*time)/w)
Iv = I(Task1["Time"],Task1["w"],m_t1.nominal_value,g,Rb_t1)
IUnc = np.array([I(Task1["Time_mit_unc"][i],Task1["w_mit_unc"][i],m_t1,g,Rb_t1).std_dev for i in range(len(Task1))])
# from the angular acceleration of a known torque
def I_3angAcc(time,m,r,h):
        return((m*r**2)*(((g*time**2)/(2*h)) -1))
I_3angAccV = I_3angAcc(Task1["Time"],m_t1.nominal_value,Rb_t1,h_t1.nominal_value)
I_3angAccUnc = np.array([I_3angAcc(Task1["Time_mit_unc"][i],m_t1,Rb_t1,h_t1).std_dev for i in range(len(Task1))])

def ITheory(m,r):
    return 0.5*(m*r*r)
ItheoryV = ITheory(m_disk,R_disk)

#from a measurement of the rotation speed
def I_3RotSpeed(angular_velocity,m,g,h,r):
    return (((2*m*g*h)/(angular_velocity**2)) - m*r**2)
I_3RotSpeedV = I_3RotSpeed(Task1['w'],m_t1.nominal_value,g,h_t1.nominal_value,Rb_t1)
I_3RotSpeedUnc = np.array([I_3RotSpeed(Task1["Time_mit_unc"][i],m_t1,g,h_t1,Rb_t1).std_dev for i in range(len(Task1))])



Inertias = [np.mean(I_3angAccV),np.mean(I_3RotSpeedV),ItheoryV.nominal_value]
Inertias_Unc = [STDOfMean(I_3angAccUnc),STDOfMean(I_3RotSpeedUnc),ItheoryV.std_dev]
InertMitUnc = [uc.ufloat(Inertias[i],Inertias_Unc[i]) for i in range(3)]
InertNames = ['AngAcc','RotSpeed',"Theory"]
BoxPlotArray = [I_3angAccV,I_3RotSpeedV,ItheoryV.nominal_value]
plt.boxplot(BoxPlotArray, showmeans=True, meanline=True,medianprops= dict(linewidth=0) ,meanprops= dict(color='darkred',linestyle='-'),
            labels=InertNames)
plt.errorbar([1.2,2.2,3.2],Inertias[:],yerr=Inertias_Unc,
             fmt='o',capsize=4,ecolor="darkred",color='darkred',elinewidth=1.3)
plt.title(r"Calculated $I_3$ via Differing Methods")
plt.ylabel(r"Moment of Inertia $kg\cdot m^2$")
TableData0 = [['',r"$I_3$",r"$\sigma$"],
            ['AngAcc',f"{Inertias[0]:.4g}" ,f"{Inertias_Unc[0]:.4g}"],
            ['RotSpeed', f"{Inertias[1]:.4g}" ,f"{Inertias_Unc[1]:.4g}"],
            ['Theory',f"{Inertias[2]:.4g}" ,f"{Inertias_Unc[2]:.4g}"]]
plt.table(cellText=TableData0,
                 loc='lower right',
                 cellLoc='center',
                 colWidths=[0.1,0.1,0.1],
                 fontsize=12,zorder=10)
plt.show()
print("Inertia3 Results")
for i in range(len(InertMitUnc)):
    print(f"Calculated Inertia from {InertNames[i]}: {InertMitUnc[i].nominal_value:.4g} ± {InertMitUnc[i].std_dev:.4g}")
print(f"mean Inertia: {np.mean(InertMitUnc[0:2]):.4g}")
    

### Task 2

2. Measure the deceleration of the gyro by friction. Determine the friction coefficient.


Friction leads to a deceleration of the gyro and therefore influences the studies on precession and
nutation. Estimate the influence of friction by measuring the damping constant of the gyro rotation.
Using the string, set the gyro into rotation, such that it reaches a rotation speed of the order of 500
rpm. Determine the decrease of the rotation speed over a period of 120 s by measuring and
protocolling the rotation speed every 10 s. Determine the damping constant from an appropriate
graph of the rotation speed against time.

In [ ]:
plt.plot(Task2['Time(s)'],Task2['Trial1_w'], label="Trial 1")
plt.plot(Task2['Time(s)'], Task2['Trial2_w'],label="Trial 2")
plt.plot(Task2['Time(s)'],Task2['Trial3_w'],label="Trial 3")
plt.title("Measured Data")
plt.xlabel("Time (s)")
plt.ylabel(r"$\omega$ $\frac{rad}{s}$")
plt.legend(loc="best")
plt.show()

To calculate the damping constant, a damped rotational motion model was used as the deceleration of the disk is soley attributal to fricitonal forces:
$$ \frac{d\omega}{dt} = -\gamma \omega $$
Solving this differential equation leads to:
$$\omega(t) = \omega_0 e^{-\gamma t}$$

Using by taking a natural log of each side a linear regression can be done on to determine $\gamma$ which is the coefficent attributal to fricitonal forces

$$ln(\omega(t)) = ln(\omega_0)-\gamma t $$

Further analysis was conducted to determine a fricitional coefficient, through the assumption that the frictional torque of the system is a result of the normal force acting on the surfrace between the disk and the ball bearing. This frictional torque also opposes the angular momentum of the system.

$$ -I\alpha = \tau_{f} =  \mu N r_{bearing}$$

where $\alpha$ is constant deceleration that is modeled by:

$$\alpha = \gamma \omega $$

The fricitonal coefficient is then:

$$\mu = \frac{\gamma I}{N r_{bearing}}$$

In [10]:
def T2ln1(t,b):
    return(np.log(Task2["Trial1_w"][0]) - b*t)
def T2ln2(t,b):
    return(np.log(Task2["Trial2_w"][0]) - b*t)
def T2ln3(t,b):
    return(np.log(Task2["Trial3_w"][0]) - b*t)
def Modelled(t,b,Trial):
        if (int(Trial) == 0):
            return(Task2["Trial1_w"][0]*np.exp(-b*t))
        if (int(Trial) == 1):
            return(Task2["Trial2_w"][0]*np.exp(-b*t))
        if (int(Trial) == 2):
            return(Task2["Trial3_w"][0]*np.exp(-b*t))


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

TitleArr = ["Trial 1","Trial 2","Trial 3"]

xArr = [Task2["Time(s)"],Task2["Time(s)"],Task2["Time(s)"]]
yArr = [Task2["Trial1_w"],Task2['Trial2_w'],Task2["Trial3_w"]]
yUncArr = [Task2["Trail1_w_unc"],Task2['Trail2_w_unc'],Task2["Trail3_w_unc"]]
ValuesArr = [NonLinAnalysis(T2ln1,Task2['Time(s)'],np.log(yArr[0]),[0.006],weights=(yUncArr[0]/yArr[0])),
             NonLinAnalysis(T2ln2,Task2["Time(s)"],np.log(yArr[1]),[0.006],weights=(yUncArr[1]/yArr[1])),
             NonLinAnalysis(T2ln3,Task2['Time(s)'],np.log(yArr[2]),[0.006],weights=(yUncArr[2]/yArr[2]))]
for i, ax in enumerate(axs):
        ax.plot(xArr[i],yArr[i], color='green',label="Measured")
        ax.plot(xArr[i],Modelled(xArr[i],ValuesArr[i][0],i),color='red',label="Fitted")
        y_lower = Modelled(xArr[i],ValuesArr[i][0],i)-[g*ValuesArr[i][1][0] for g in yArr[i]]
        y_upper = Modelled(xArr[i],ValuesArr[i][0],i)+[g*ValuesArr[i][1][0] for g in yArr[i]]
        ax.fill_between(xArr[i],y_lower,y_upper,color="black",alpha=0.6, label=f"Fitted Uncertainty")
        TableData = [[r'$b$',f"{ValuesArr[i][0][0]:.4f}"],
        ['std', f"{STDOfMean(np.array([g*ValuesArr[i][1][0] for g in yArr[i]])):.4f}"],
        ['r²', f"{ValuesArr[i][2]:.4f}"]]
        ax.table(cellText=TableData,
                 loc='lower left',
                 cellLoc='center',
                 colWidths=[0.15, 0.15],
                 fontsize=12,zorder=10)
        ax.set_xlabel("Time (s)",fontsize=8)
        ax.set_ylabel(r"$\omega$ $\frac{rad}{s}$")
        ax.legend(loc="best",fontsize=7)
        ax.set_title(TitleArr[i])

plt.suptitle("Damping Constant",fontsize=12)
plt.tight_layout()
plt.show()

gamma = np.mean([ValuesArr[0][0][0],ValuesArr[1][0][0],ValuesArr[2][0][0]])
gamma_unc = STDOfMean(np.array([ValuesArr[0][1][0],ValuesArr[1][1][0],ValuesArr[2][1][0]]))
print(f"Damping Constant: {gamma:.6f} ± {gamma_unc:.6f}")


### Task 3


3. Measure the precession frequency as a function of the rotation frequency of the gyro disk for two
different torque values. Plot the precession frequency in an appropriate manner as a function of the
rotation frequency and determine the value of the moment of inertia component I3


The fixation of the gyro to the tripod stand is removed and with the use of the counterweights the
gyro axis is brought into equilibrium. Check that the axis is in indifferent equilibrium for arbitrary
values of its inclination with respect to the horizontal.
Wind the free gyro up while holding the gyro axis fixed. Hook the additional weight onto the screw at
the end of the gyro axis opposite to the rotating disk and let the gyro go. Measure the time for half a
precession period with the stopwatch; both at the start and at the end of the half precession period
measure the rotation speed of the gyro disk. Use the average value of the rotation speeds as rotation
frequency.
Perform the measurements for ten different rotation frequencies in the range between about 200
and 600 rpm and for weights of 47 g and 94 g.
Plot the precession frequency in an appropriate way as a function of the rotation frequency and
compare to theory.

In [ ]:
N = 9.81*m_disk
gamun = uc.ufloat(gamma,gamma_unc)
InertUnc1 = uc.ufloat(Inertias[0],Inertias_Unc[0])
InertUnc2 = uc.ufloat(Inertias[1],Inertias_Unc[1])
InertUnc3 = uc.ufloat(Inertias[2],Inertias_Unc[2])
PrinArr= [r"Energy I₃", r"Rotational Speed I₃",r"Theoretical I₃"]
muarr = []
for i,I in enumerate([InertUnc1,InertUnc2,InertUnc3]):
    mu = (gamma*I)/(N*Rbear*Task2["Trial1_w"])
    muarr.append(mu)
    # print(f"Friction Coefficient: {mu.nominal_value:.6f} ± {mu.std_dev:.6f} from "+ PrinArr[i])

muarr0_nom,muarr0_unc = np.array(UnpackUncArray(muarr[0]))
muarr1_nom,muarr1_unc = np.array(UnpackUncArray(muarr[1]))
muarr2_nom,muarr2_unc = np.array(UnpackUncArray(muarr[2]))
muarr_nom = np.array([muarr0_nom,muarr1_nom,muarr2_nom])
muarr_unc = np.array([muarr0_unc,muarr1_unc,muarr2_unc])

list=[r"\alpha",r"\omega",r"T"]
list2=["#B9A1D6","#70B8B2","#FF6F61"]
# fig, axs = plt.subplots(3, 1, figsize=(6, 4))
# for i, ax in enumerate(axs):
#     ax.fill_between(Task2["Time(s)"],muarr_nom[i]-muarr_unc[i],muarr_nom[i]+muarr_unc[i],alpha=0.3)
#     ax.plot(Task2["Time(s)"],muarr_nom[i],label=(f"$\mu^{list[i]}$"))
#     ax.set_xlabel("Time (s)",fontsize=8)
#     ax.set_ylabel(r"$\mu$")
#     ax.legend(loc="best")
#     ax.set_title(f"$\mu^{list[i]}$ Determined from $I₃^{list[i]}$")


for i in range(3):
    plt.fill_between(Task2["Time(s)"],muarr_nom[i]-muarr_unc[i],muarr_nom[i]+muarr_unc[i],alpha=0.3,color=list2[i])
    plt.plot(Task2["Time(s)"],muarr_nom[i],label=(f"$\mu^{list[i]}$"),color=list2[i])

plt.title(f"$\mu$ Determined from $I₃$'s")
plt.legend(loc="best")
plt.xlabel("Time (s)",fontsize=8)
plt.ylabel(r"$\mu$ (a.u.)")
plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))  # Create a figure with 1 row and 2 columns

# First subplot
for i in range(3):
    ax1.fill_between(Task2["Time(s)"], muarr_nom[i] - muarr_unc[i], muarr_nom[i] + muarr_unc[i], 
                     alpha=0.3, color=list2[i])
    ax1.plot(Task2["Time(s)"], muarr_nom[i], label=f"$\mu^{list[i]}$", color=list2[i])

ax1.set_title("$\mu$ Determined from $I₃$'s (vs Time)")
ax1.legend(loc="best")
ax1.set_xlabel("Time (s)", fontsize=8)
ax1.set_ylabel(r"$\mu$ (a.u.)")

# Second subplot
for i in range(3):
    ax2.fill_between(Task2["Trial1_w"], muarr_nom[i] - muarr_unc[i], muarr_nom[i] + muarr_unc[i], 
                     alpha=0.3, color=list2[i])
    ax2.plot(Task2["Trial1_w"], muarr_nom[i], label=f"$\mu^{list[i]}$", color=list2[i])

ax2.set_title("$\mu$ Determined from $I₃$'s (vs $\omega$)")
ax2.legend(loc="best")
ax2.set_xlabel(r"$\omega$ $\frac{rad}{s}$", fontsize=8)
ax2.set_ylabel(r"$\mu$ (a.u.)")

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()


In [ ]:
Torque1 = Rw_t3*g*0.047
Torque2 = Rw_t3*g*0.094 
avgm1w = np.array([(np.mean(np.array([Task3.iloc[i,9],Task3.iloc[i,12]]))) for i in range(len(Task3["m1w0"])-1)])
avgm2w = np.array([(np.mean(np.array([Task3.iloc[i,17],Task3.iloc[i,20]]))) for i in range(len(Task3["m2w0"]))])
RotationalFrequencym1 = avgm1w/2*np.pi
PrecessionFrequencym1 = 1/(2*Task3["Time1_mit_unc"][0:11])
RotationalFrequencym2 = avgm2w/2*np.pi
PrecessionFrequencym2 = 1/(2*Task3["Time2_mit_unc"])

#Sorting Numpy Arrays
RotationalFrequencym1, PrecessionFrequencym1 = SortByfirst(RotationalFrequencym1,PrecessionFrequencym1)
RotationalFrequencym2, PrecessionFrequencym2 = SortByfirst(RotationalFrequencym2,PrecessionFrequencym2)

x1,x1unc = UnpackUncArray(RotationalFrequencym1)
x1 = 1/np.array(x1)
x1unc=(np.array(x1unc)/(np.array(x1)*np.array(x1)))
y1,y1unc = UnpackUncArray(PrecessionFrequencym1)
x2,x2unc = UnpackUncArray(RotationalFrequencym2)
x2 = 1/np.array(x2)
x2unc=(np.array(x2unc)/(np.array(x2)*np.array(x2)))
y2,y2unc = UnpackUncArray(PrecessionFrequencym2)
fig, axs = plt.subplots(1, 2, figsize=(16, 6))
for i in range(len(axs)):
                
        axs[i].errorbar(x1,y1,yerr=y1unc,fmt='o')
        axs[i].scatter(x1,y1,s=10,color="lightblue",edgecolors='k',linewidth=0.5,zorder=10,label = "Torque 1")
        m1p,m1cov = np.polyfit(x1,y1,1,cov=True,w=(1/np.array(y1unc)))
        m1std = np.sqrt(np.diag(m1cov))
        axs[i].plot(x1,np.polyval(m1p,x1))
        m1r = R2(y=y1,y_pred=np.polyval(m1p,x1))
        axs[i].errorbar(x2,y2,yerr=y2unc,fmt='o')
        axs[i].scatter(x2,y2,s=10,color="darkorange",edgecolors='k',linewidth=0.5,zorder=10,label = "Torque 2")
        m2p,m2cov = np.polyfit(x2,y2,1,cov=True,w=(1/np.array(y2unc)))
        m2std = np.sqrt(np.diag(m2cov))
        axs[i].plot(x2,np.polyval(m2p,x2))
        m2r = R2(y=y2,y_pred=np.polyval(m2p,x2))

        axs[i].set_xlabel(r"$\frac{1}{Rotation Frequency} (s)$")
        axs[i].set_ylabel("Precession Frequency (Hz)")
        if i == 1:
                axs[i].fill_between(x2,np.polyval(m2p,x2)-m2std[0],np.polyval(m2p,x2)+m2std[0],
                                    color="gray",alpha=0.3,label="Torque 2 Uncertainty")
                axs[i].fill_between(x1,np.polyval(m1p,x1)-m1std[0],np.polyval(m1p,x1)+m1std[0],
                                    color="lightblue",alpha=0.5,label="Torque 1 Uncertainty")
                axs[i].set_title("Precession vs Rotational Frequencies w/ Uncertainty")
        else:
                axs[i].set_title("Precession vs Rotational Frequencies")
                T3_TableData = [["",r'Torque 1',r"Torque 2"],
                        ['p', f"{m1p[0]:.4f}",f"{m2p[0]:.4f}"],
                        [r'$\sigma$', f"{m1std[0]:.4f}",f"{m2std[0]:.4f}"],
                        ['r²', f"{m1r:.4f}",f"{m2r:.4f}"]]
                axs[i].table(cellText=T3_TableData,loc='center right',cellLoc='center',colWidths=[0.1, 0.1,0.1],zorder=10)
        axs[i].legend()

plt.show()
print(f"Torque 1 - p: {m1p[0]:.4g} ± {m1std[0]:.4g}")
print(f"Torque 2 - p: {m2p[0]:.4g} ± {m2std[0]:.4g}")

$$\Omega_p = \frac{M}{L} = \frac{m_z g z_z}{I_3 \omega} = \frac{p}{\omega}$$

So


$$ p = \frac{m_z g z_z}{I_3} \implies I_3 = \frac{m_z g z_z}{p}$$

In [ ]:
def T3_I_3(p,Torque):
    return(Torque/p)
I3_Torq1 = T3_I_3(uc.ufloat(m1p[0],m1std[0]),Torque1)
I3_Torq2 = T3_I_3(uc.ufloat(m2p[0],m2std[0]),Torque2)
print(f"Torque 1 I3: ({I3_Torq1.nominal_value:.4g} ± {I3_Torq1.std_dev:.4g})Kg m^2")
print(f"Torque 2 I3: ({I3_Torq2.nominal_value:.4g} ± {I3_Torq2.std_dev:.4g})Kg m^2\n")
print("Comparision")
for i in range(len(InertMitUnc)):
    print(f"Calculated Torque from {InertNames[i]}: {InertMitUnc[i].nominal_value:.4g} ± {InertMitUnc[i].std_dev:.4g}")

### Task 4

4. Measure the frequency of nutation as a function of the rotation frequency of the gyro disk. Plot
the nutation frequency in an appropriate manner as a function of the rotation frequency and
determine the value of the moment of inertia component I1.


The free gyro is brought into equilibrium by adjusting the counter weights and is wound up.
Subsequently, a nutation is generated by a brief vertical impulse on the gyro axis. The time for three
nutation periods is measured with the stopwatch and the rotation speed is recorded at the start and
end. Use the average value of the rotation speeds as rotation frequency. These measurements are
performed for ten different rotation frequencies in the range between about 150 and 500 rpm.
Plot the nutation frequency in an appropriate way as a function of the rotation frequency of the gyro.
Determine the value of the component I1 of the moment of inertia tensor

In [ ]:
Task4["avgRotFreq"] = np.array([(np.mean(np.array([Task4.iloc[i,6],Task4.iloc[i,8]]))) for i in range(len(Task4["Try"]))])
Task4["avgRotFreq"] = (1/(2*np.pi))*Task4["avgRotFreq"]
Task4.sort_values("avgRotFreq")

In [ ]:
omit_indices = {3, 5, 6}
avgRotFreq = Task4["avgRotFreq"].drop(omit_indices).reset_index(drop=True)
avgRotFreq_nom, avgRotFreq_Unc = UnpackUncArray(avgRotFreq)
NutateFreq = 3/np.array(Task4["Time_mit_unc"].drop(omit_indices).reset_index(drop=True))
NutateFreq_nom, NutateFreq_Unc = UnpackUncArray(NutateFreq)
alphs = Task4["α(degrees)"].drop(omit_indices).reset_index(drop=True)



plt.scatter(avgRotFreq_nom,NutateFreq_nom,s=10,color="olive",edgecolors='k',linewidth=0.5,zorder=10,label = "Measured Data")
t4p,t4cov = np.polyfit(avgRotFreq_nom,NutateFreq_nom,1,cov=True)
t4std = np.sqrt(np.diag(t4cov))
plt.errorbar(avgRotFreq_nom,NutateFreq_nom,NutateFreq_Unc,fmt='o',color="green")
fitted = np.polyval(t4p,avgRotFreq)
fitted_nom, fitted_Unc = np.array(UnpackUncArray(fitted))
plt.plot(avgRotFreq_nom,fitted_nom,label="Fitted")
sort = np.array(avgRotFreq_nom)[np.argsort(avgRotFreq_nom)]
plt.fill_between(sort,np.polyval(t4p,sort)-t4std[0],np.polyval(t4p,sort)+t4std[0],
                    color="lightblue",alpha=0.5,label="Fitted Uncertainty")
t4rr = R2(NutateFreq_nom,fitted_nom)
plt.xlabel(r"Rotation Frequency $Hz$")
plt.ylabel(r"Nutation Frequency $Hz$")
plt.title("Nutation vs Rotation Frequency")
TableData3 = [[r'$\xi$',f"{t4p[0]:.4g}"],
[r'$\sigma$', f"{t4std[0]:.4g}"],
['r²', f"{t4rr:.4}"]]
plt.table(cellText=TableData3,
                 loc='lower right',
                 cellLoc='center',
                 colWidths=[0.1, 0.1],
                 fontsize=12,zorder=10)
plt.legend(loc="upper left")
plt.show()
                                                                        

In [ ]:
I1s = []
for _, I in enumerate(InertMitUnc):
    intere = I/(1+uc.ufloat(t4p[0],t4std[0]))
    print(f"Inertia 1 from Inertia {InertNames[_]}: {np.mean(intere):.4g}")
    I1s.append(intere)
print()
wpers= []
wp22 = []
for _ in range(len(InertMitUnc)):
    wper = np.array((InertMitUnc[_]/(avgRotFreq*np.tan(alphs)))/I1s[_])
    print(f"omega perp from Inertia {InertNames[_]}: {np.mean(wper):.8g}")
    wpers.append(wper)
    RotAngFreq = (np.array(avgRotFreq)/(2*np.pi))
    frenutat = (1/(2*np.pi))*(sqrt(np.mean(RotAngFreq)**2 + (np.mean(wper)/(2*np.pi))**2))
    print(f"Fp for Inertia {InertNames[_]}: {np.mean(frenutat):.8g}")
    print()

In [ ]:
plt.errorbar([1,2,3],[I1s[0].nominal_value,I1s[1].nominal_value,I1s[2].nominal_value],yerr=[I1s[0].std_dev,I1s[1].std_dev,I1s[2].std_dev],
             fmt='o',capsize=4,ecolor="darkred",color='darkred',elinewidth=1.3)
plt.scatter([1,2,3],[I1s[0].nominal_value,I1s[1].nominal_value,I1s[2].nominal_value],s=10,color="olive",edgecolors='k',linewidth=0.5,zorder=10,label = "Measured Data")
plt.title(r"Calculated $I_1$ via Differing Methods")
plt.xticks([1, 2, 3], InertNames)
plt.ylabel(r"Moment of Inertia $kg\cdot m^2$")
TableData3 = [['',r"$I_1$",r"$\sigma$"],
            ['AngAcc',f"{I1s[0].nominal_value:.4g}" ,f"{I1s[0].std_dev:.4g}"],
            ['RotSpeed', f"{I1s[1].nominal_value:.4g}" ,f"{I1s[1].std_dev:.4g}"],
            ['Theory',f"{I1s[2].nominal_value:.4g}" ,f"{I1s[2].std_dev:.4g}"]]
plt.table(cellText=TableData3,
                 loc='lower right',
                 cellLoc='center',
                 colWidths=[0.1,0.1,0.1],
                 fontsize=12,zorder=10)
plt.show()